In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import os
from urllib.parse import urlsplit

def parse_index_page(index_url):
    resp = requests.get(index_url)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser")

    result_list = []
    table = soup.find("table")
    if not table:
        print("テーブルが見つかりません。HTML構造を確認してください。")
        return result_list

    for row in table.find_all("tr"):
        cols = row.find_all("td")
        # 今回は4列 (英名, 日名, ダウンロード, URL) を想定
        if len(cols) < 4:
            continue
        guide_en = cols[0].get_text(strip=True)
        guide_jp = cols[1].get_text(strip=True)
        # cols[2] は "DOC - PDF" など不要なら無視してOK
        base_url = cols[3].get_text(strip=True)

        result_list.append({
            "GuideNameEn": guide_en,
            "GuideNameJp": guide_jp,
            "BaseURL": base_url
        })
    return result_list

def remove_newlines_and_dots(text):
    if not text:
        return ""
    text = re.sub(r"[\r\n]+", "", text)
    text = re.sub(r"[.…]+", "", text)
    return text.strip()

def parse_toc_from_url(url, doc_name, guide_en, guide_jp):
    """
    doc_name, guide_en, guide_jpを使って、
    url上で #__RefHeading__ や #_bookmark のアンカーを探し →
    (SectionTitle1, SectionTitle2, AnchorID, PageNumberなど)を返す
    """
    resp = requests.get(url)
    if resp.status_code != 200:
        print(f"HTTPエラー: {resp.status_code}")
        return []
    soup = BeautifulSoup(resp.text, "html.parser")

    # ＜修正箇所＞アンカーIDのパターンを拡張
    # startswith("#__RefHeading__") に加え、startswith("#_bookmark") などを拾う
    toc_links = []
    for a in soup.find_all("a"):
        href = a.get("href", "")
        # 例: #__RefHeading___TocXXXX, #_bookmark0, #_bookmark1 ...
        if (href.startswith("#__RefHeading__")
            or href.startswith("#_bookmark")
            or href.startswith("#_Toc")):
            toc_links.append(a)

    print(f"URL={url} から {len(toc_links)} 個のTOCリンクを検出")

    records = []
    lastSection1 = ""

    for link in toc_links:
        raw_href = link.get("href","")
        # 例: #__RefHeading___Toc485730093 or #_bookmark0
        # '#' を取り除いて実際の AnchorID を取り出す
        anchor_id = raw_href.lstrip("#")

        raw_text = link.get_text()

        # 末尾の数字をページ番号として取り出すかどうか
        match_page = re.search(r"(\d+)$", raw_text)
        if match_page:
            page_num = match_page.group(1)
            raw_text = re.sub(r"\s*\d+$", "", raw_text)
        else:
            page_num = ""

        text_val = remove_newlines_and_dots(raw_text)

        # "セクション X:" の判定
        match_sec = re.match(r"^(セクション\s*\d+)\s*:\s*(.*)$", text_val)
        if match_sec:
            section1 = remove_newlines_and_dots(match_sec.group(1))
            section2 = remove_newlines_and_dots(match_sec.group(2))
            lastSection1 = section1
        else:
            # 直近のセクション1を流用
            section1 = lastSection1
            section2 = text_val

        # full_link
        full_link = f"{url}{raw_href}"

        rec = {
            "DocName": doc_name,
            "GuideNameEn": guide_en,
            "GuideNameJp": guide_jp,
            "SectionTitle1": section1,
            "SectionTitle2": section2,
            "PageNumber": page_num,
            "AnchorID": anchor_id,
            "BaseURL": url,
            "FullLink": full_link,
            "Remarks": text_val
        }
        records.append(rec)
    return records

def update_mapping_db(mapping_csv_path, new_records, base_url):
    if os.path.exists(mapping_csv_path):
        df = pd.read_csv(mapping_csv_path, encoding='utf-8-sig')
        print(f"{mapping_csv_path} 読み込み: {len(df)} 行")
    else:
        df = pd.DataFrame(columns=[
            "DocName","GuideNameEn","GuideNameJp","SectionTitle1","SectionTitle2",
            "PageNumber","AnchorID","BaseURL","FullLink","Remarks"
        ])
        print(f"{mapping_csv_path} が無いので新規作成")

    before_count = len(df)
    # 同じBaseURLの行を削除 (既存データをアップデートするイメージ)
    df = df[df["BaseURL"] != base_url]
    after_count = len(df)
    print(f"BaseURL='{base_url}' の行を {before_count - after_count}件 削除")

    new_df = pd.DataFrame(new_records)
    # 結合
    df = pd.concat([df, new_df], ignore_index=True)

    df.to_csv(mapping_csv_path, index=False, encoding='utf-8-sig')
    print(f"{mapping_csv_path} を更新 (最終行数={len(df)})")

def main():
    mapping_csv = "mapping_DB.csv"
    index_url = "https://la-concur-helper.github.io/concur-docs/index.htm"

    print(f"Indexページ: {index_url} を解析中...")
    index_list = parse_index_page(index_url)
    if not index_list:
        print("index_listが空です。終了します。")
        return

    # ユーザに どのdocを処理するか 選択してもらう
    print("\n=== 現在インデックスにあるドキュメント一覧 ===")
    for i, info in enumerate(index_list, start=1):
        print(f"{i}. {info['GuideNameEn']} / {info['GuideNameJp']} => {info['BaseURL']}")

    sel = input("\n何番のドキュメントを処理しますか？番号を入力(Enterで終了): ")
    if not sel.strip():
        print("キャンセル。終了。")
        return
    sel_idx = int(sel)

    if sel_idx < 1 or sel_idx > len(index_list):
        print("不正な番号です。終了。")
        return

    chosen = index_list[sel_idx - 1]
    guide_en = chosen["GuideNameEn"]
    guide_jp = chosen["GuideNameJp"]
    base_url = chosen["BaseURL"]

    # ＜修正箇所＞ doc_name を .html/.htm どちらにも対応させて .docx に置換
    fname = os.path.basename(urlsplit(base_url).path)  # e.g. "Exp_SG_Account_Codes-jp.html"
    # ".html" も ".htm" もまとめて ".docx" にする
    doc_name = re.sub(r"\.html?$", ".docx", fname, flags=re.IGNORECASE)

    print(f"\n--- 選択ドキュメント ---\nDocName: {doc_name}\nGuideNameEn: {guide_en}\nGuideNameJp: {guide_jp}\nBaseURL: {base_url}\n")

    # 目次解析
    records = parse_toc_from_url(base_url, doc_name, guide_en, guide_jp)
    if not records:
        print("目次が空です。終了。")
        return

    # DB更新
    update_mapping_db(mapping_csv, records, base_url)
    print("処理が完了しました。")


if __name__ == "__main__":
    main()


Indexページ: https://la-concur-helper.github.io/concur-docs/index.htm を解析中...

=== 現在インデックスにあるドキュメント一覧 ===
1. Guide Name / ガイド名 => URL
2. Account Codes / 勘定科目コード(2017年01月10日版) => https://la-concur-helper.github.io/concur-docs/Exp_SG_Account_Codes-jp.html
3. Allocations / 配賦(2020年7月1日版) => https://la-concur-helper.github.io/concur-docs/Exp_SG_Allocations-jp.html
4. Attendees / 同席者(2023年6月28日版) => https://la-concur-helper.github.io/concur-docs/Exp_SG_Attendees-jp.html
5. Audit Rules / 監査ルール(2024年8月17日版)別冊:日付の演算子について => https://la-concur-helper.github.io/concur-docs/Exp_SG_Audit_Rules-jp.html
6. Audit Rules (Validation Rules) / 監査ルール（検証ルール）(2022年8月5日版) => https://la-concur-helper.github.io/concur-docs/Exp_SG_Audit_Rules_Validation_Rules-jp.html
7. Budget (Shared) / 予算（製品共通）(2021年2月20日版)別冊:予算管理についてのよくある質問(2019年5月29日版) => https://la-concur-helper.github.io/concur-docs/Shr_SG_Budget-jp.html
8. Car Configuration / 車両設定(2021年1月27日版) => https://la-concur-helper.github.io/concur-docs/Exp_SG_Car_Con